In [1]:
import os

os.chdir("C:/Users/flexmatch/Desktop/ssom/code/3.SNS-categorizer/src")

In [2]:
from datetime import datetime
import boto3
from io import BytesIO
from dotenv import load_dotenv

import pandas as pd

load_dotenv('C:/Users/flexmatch/Desktop/ssom/code/3.SNS-categorizer/config/.env')
aws_access_key = os.getenv("aws_accessKey")
aws_secret_key = os.getenv("aws_secretKey")
bucket_name = 'flexmatch-data'
region_name = 'ap-northeast-2'

s3 = boto3.client('s3',
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_key,
                region_name=region_name)

today = datetime.now()
year, week, _ = today.isocalendar()

# profile_data_prefix = f'instagram-data/tables/EXTERNAL_2_RECENT_USER_INFO_MTR/year={year}/week={week}'
# media_data_prefix = f'instagram-data/tables/EXTERNAL_2_BY_USER_ID_MEDIA_DTL_INFO/year={year}/week={week}'

profile_data_prefix = f'instagram-data/tables/RECENT_USER_INFO_MTR/year={year}/week={week}/'
media_data_prefix = f'instagram-data/tables/BY_USER_ID_MEDIA_DTL_INFO/year={year}/week={week}/'

# profile 파일 목록
profile_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=profile_data_prefix)
profile_keys = [obj['Key'] for obj in profile_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

# media 파일 목록
media_response = s3.list_objects_v2(Bucket=bucket_name, Prefix=media_data_prefix)
media_keys = [obj['Key'] for obj in media_response.get('Contents', []) if obj['Key'].endswith('.parquet')]

profile_dfs = []

for key in profile_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    profile_dfs.append(df)

not_conn_user_info_mtr = pd.concat(profile_dfs, ignore_index=True)

media_dfs = []

for key in media_keys:
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    body = obj['Body'].read()
    df = pd.read_parquet(BytesIO(body))
    media_dfs.append(df)

not_conn_user_media_info = pd.concat(media_dfs, ignore_index=True)

In [3]:
from DB_connection import *
from model_inference import *

c:\Users\flexmatch\AppData\Local\pypoetry\Cache\virtualenvs\sns-categorizer--w_Xk74Y-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# pip install "paramiko<3.4" --force-reinstall

In [5]:
flexmatch_influencer_info, conn_user_info_mtr, conn_user_media_info = get_all_infos()

DB 접속 성공
DB 접속 성공
DB 접속 성공


In [7]:
conn_user_info_mtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   acnt_id                 38 non-null     int64 
 1   acnt_nm                 38 non-null     object
 2   web_addr                31 non-null     object
 3   acnt_sub_nm             35 non-null     object
 4   intro_txt               35 non-null     object
 5   profile_photo_url_addr  36 non-null     object
 6   acnt_conn_yn            38 non-null     object
 7   category_nm             24 non-null     object
 8   follower_cnt            38 non-null     int64 
 9   follow_cnt              38 non-null     int64 
 10  media_cnt               38 non-null     int64 
dtypes: int64(4), object(7)
memory usage: 3.4+ KB


In [8]:
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].astype('Int64')
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].fillna(0)
# flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].astype(int)
# flexmatch_influencer_info

flexmatch_influencer_info['member_uid'] = flexmatch_influencer_info['member_uid'].fillna(0).astype(int)
flexmatch_influencer_info

,user_id,member_uid,add1
0,admin,0,13F80C4C43DC63E772650995CA18639F
1,test901,5180,su.jin.1995
2,tkddmlwns^^^DD,0,oddity
3,ssh8918,0,https://www.instagram.com/flex.item/?hl=
4,yong,0,https://www.instagram.com/su.jin.1995/
...,...,...,...
1799,gksmfdkdud@naver.com,13559,oil.hwa
1800,sungkyung1031@kakao.com,13563,ksxxida
1801,reborncosmetics@kakao.com,0,danika86_
1802,tomokioi030@naver.com,13580,0o0_kikii


In [28]:
not_conn_user_info_mtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   acnt_id                 1015 non-null   object
 1   acnt_nm                 1015 non-null   object
 2   web_addr                551 non-null    object
 3   acnt_sub_nm             982 non-null    object
 4   intro_txt               939 non-null    object
 5   profile_photo_url_addr  1012 non-null   object
 6   acnt_conn_yn            1015 non-null   object
 7   category_nm             1012 non-null   object
 8   follower_cnt            1015 non-null   int64 
 9   follow_cnt              1015 non-null   int64 
 10  media_cnt               1015 non-null   int64 
dtypes: int64(3), object(8)
memory usage: 87.4+ KB


In [30]:
not_conn_user_media_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47736 entries, 0 to 47735
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   acnt_id             47736 non-null  object
 1   media_id            47736 non-null  object
 2   media_type_nm       47736 non-null  object
 3   reels_feed_type_nm  47736 non-null  object
 4   media_url_addr      47643 non-null  object
 5   media_unq_url_addr  47736 non-null  object
 6   tmnl_url_addr       17127 non-null  object
 7   reg_dt              47736 non-null  object
 8   media_cn            47088 non-null  object
 9   acnt_conn_yn        47736 non-null  object
 10  feed_share_yn       0 non-null      object
 11  cmnt_actvtn_yn      0 non-null      object
dtypes: object(12)
memory usage: 4.4+ MB


In [9]:
flexmatch_influencer_info['add1'] = flexmatch_influencer_info['add1'].str.replace('https://www.instagram.com/', '')

In [10]:
flexmatch_influencer_info['acnt_nm'] = flexmatch_influencer_info['add1'].str.replace('/', '')

In [30]:
new_profile_data = pd.concat([not_conn_user_info_mtr, conn_user_info_mtr], axis=0)
new_media_data = pd.concat([not_conn_user_media_info, conn_user_media_info], axis=0)

In [31]:
category_labels = ['IT', '게임', '결혼/연애', '교육', '다이어트/건강보조식품', '만화/애니/툰', '문구/완구', '미술/디자인', '반려동물', '베이비/키즈', '뷰티', '브랜드공식계정',
                    '사진/영상', '셀럽', '스포츠', '시사', '엔터테인먼트', '여행/관광', '유명장소/핫플', '일상', '자동차/모빌리티', '짤/밈', '취미', '패션', '푸드', '홈/리빙']

merged_df, predict_df = tokenize_and_predict_batch(new_profile_data, new_media_data, category_labels)
merged_df = merged_df[['acnt_id', 'acnt_nm']].reset_index(drop=True)
predict_df.reset_index(drop=True, inplace=True)

# final data after category labeling
final_predict_df = pd.concat([merged_df, predict_df], axis=1)

Map: 100%|██████████| 76324/76324 [00:10<00:00, 7619.09 examples/s]


In [32]:
final_predict_df['acnt_id'].nunique()

845

In [33]:
db_merge_df = pd.merge(final_predict_df, flexmatch_influencer_info, on='acnt_nm', how='left')

main_category = db_merge_df.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].agg(lambda x: x.value_counts().idxmax()).to_frame().reset_index().rename(columns={'bert_top_label' : 'main_category'})

top_3_labels = db_merge_df.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].value_counts().groupby(level=[0,1]).head(3).reset_index(name='count')
top_3_labels_joined = (top_3_labels.groupby(['acnt_id', 'acnt_nm'])['bert_top_label'].apply(lambda x: '@'.join(x)).reset_index(name='top_3_category'))

top_3_labels_joined = top_3_labels_joined.drop(columns=['acnt_id', 'acnt_nm'])
final_df = pd.concat([main_category, top_3_labels_joined], axis=1)

In [34]:
final_df

,acnt_id,acnt_nm,main_category,top_3_category
0,17841400189532541,24oldfency,뷰티,뷰티@엔터테인먼트@패션
1,17841400262764952,freesencer,여행/관광,여행/관광@엔터테인먼트@일상
2,17841400360358101,bro_bums,일상,일상@스포츠@여행/관광
3,17841400539004334,bbangsori,푸드,푸드@뷰티@여행/관광
4,17841400608483053,danika86_,교육,교육@베이비/키즈@여행/관광
...,...,...,...,...
840,17841475247014767,en_ho_an,베이비/키즈,베이비/키즈
841,17841475396295656,daily.haru4,일상,일상@유명장소/핫플@베이비/키즈
842,17841475402483783,wonni_home_life,홈/리빙,홈/리빙
843,17841475482062860,e_hacong,일상,일상@결혼/연애@유명장소/핫플
